# Locate Hunza pixels

Figure out lat/lon locations of Joan's pixels of interest in the Hunza

In [ ]:
%pylab notebook
from cetbtools.ease2conv import Ease2Transform
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import numpy as np

Read a UIB cube to get projected and lat/lon coordinates.  
Any cube at a given resolution will do, they are all the same.
Use the UL corner of the cube to find out its corresponding offset from the 
full hemisphere grids.

In [ ]:
def find_UIB_cube_offset(cubeType=None, verbose=False):
    if not cubeType:
        cubeType = '18H-GRD'
        
    cubeFile = "%s%s%s%s" % (
        "/Users/brodzik/cetb_data/AQUA_AMSRE/N/cubes_UIB/",
        "CETB.cubefile.UIB.AQUA_AMSRE-",
        cubeType,
        "-RSS-v1.2.2003.TB.nc")
        
    f = Dataset(cubeFile, "r", "NETCDF4")   
    lats = f.variables['latitude'][:]
    lons = f.variables['longitude'][:]
    baseGPD = f.variables['crs'].long_name
    f.close()
    
    # find and return the baseGPD (row, col) offset for cubeUL(0, 0) location
    grid = Ease2Transform(baseGPD)
    row_offset, col_offset = grid.geographic_to_grid(lats[0,0], lons[0,0])

    if verbose:
        print("%10s offsets for cubeType=%s" % (baseGPD, cubeType))
        print("(Add these offsets to cube (row, col) to get row, col in full hemisphere)")
        print("offset row = %f, offset col = %f" % (row_offset, col_offset))
        
    return row_offset, col_offset    

In [ ]:
types = ['18H-GRD', '18H-SIR', '36H-SIR']
for thisType in types:
    offset_row, offset_col = find_UIB_cube_offset(cubeType=thisType, verbose=True)

In [ ]:
cubeFile = "CETB.cubefile.UIB.AQUA_AMSRE-18H-GRD-RSS-v1.2.2003.TB.nc"
f = Dataset(cubeFile, "r", "NETCDF4")
x = f.variables['x'][:]
y = f.variables['y'][:]
lats = f.variables['latitude'][:]
lons = f.variables['longitude'][:]
print(f.variables['crs'].long_name)
f.close()

Find out the subset UL corner geographic location and figure out 
which full hemisphere cell this corresponds to

In [ ]:
# Figures out the UIB cube subset row,col coordinates at a given lat,lon
def grid_locations_of(lat, lon):
    gpds = ["EASE2_N3.125km", "EASE2_N6.25km", "EASE2_N25km"]
    types = ["36H-SIR", "18H-SIR", "18H-GRD"]
    
    print("Input lat, lon = %.6f, %.6f" % (lat, lon))
    rows = np.zeros((3))
    cols = np.zeros((3))
    for i, (thisGpd, thisType) in enumerate(zip(gpds, types)):
        grid = Ease2Transform(thisGpd)
        row, col = grid.geographic_to_grid(lat, lon)
        print("%15s         : row, col = %.6f, %.6f" % (thisGpd, row, col))
        
        # Get the cube offsets
        offset_row, offset_col = find_UIB_cube_offset(
            cubeType=thisType)
        UIBrow = row - offset_row
        UIBcol = col - offset_col
        print("%15s(%s): row, col = %.6f, %.6f" % ("UIB", thisType, UIBrow, UIBcol))
        rows[i] = UIBrow
        cols[i] = UIBcol

    rows = (rows + 0.5).astype('int32')
    cols = (cols + 0.5).astype('int32')

    print(gpds)
    print("%d %d %d %d %d %d" % (
        cols[0], rows[0],
        cols[1], rows[1],
        cols[2], rows[2]))
    
    
    return (rows, cols)


In [ ]:
subULlat, subULlon = 39.461846, 82.092837
grid_locations_of(subULlat, subULlon)


In [ ]:
Gilgitlat, Gilgitlon = 35.9261, 74.3711
grid_locations_of(Gilgitlat, Gilgitlon)

In [ ]:
Naltarlat, Naltarlon = 36.1667, 74.1833
grid_locations_of(Naltarlat, Naltarlon)

In [ ]:
# Ziarat
lat, lon = 36.798, 74.482
grid_locations_of(lat, lon)

In [ ]:
# Khunjerab
lat, lon = 36.8411,75.4192
grid_locations_of(lat, lon)

In [ ]:
def find_UIB_cube_geographic_at(row, col, cubeType=None, verbose=False):
    if not cubeType:
        cubeType = '36H-SIR'
        
    cubeFile = "%s%s%s%s" % (
        "/Users/brodzik/cetb_data/AQUA_AMSRE/N/cubes_UIB/",
        "CETB.cubefile.UIB.AQUA_AMSRE-",
        cubeType,
        "-RSS-v1.2.2003.TB.nc")
        
    f = Dataset(cubeFile, "r", "NETCDF4")   
    lats = f.variables['latitude'][:]
    lons = f.variables['longitude'][:]
    baseGPD = f.variables['crs'].long_name
    f.close()
    
    if verbose:
        print("base=%10s for cubeType=%s" % (baseGPD, cubeType))
        print("row, col = %f, %f" % (row, col))
        print("lat, lon = %f, %f" % (lats[row, col], lons[row, col]))
        
    return (lats[row, col], lons[row, col])

In [ ]:
# Melt Onset pixels (in 3 km grid)
pixel_cols = np.array([39, 30, 65, 61, 57, 50])
pixel_rows = np.array([256, 253, 217, 243, 243, 254])
for (row, col) in zip(pixel_rows, pixel_cols):
    lat, lon = find_UIB_cube_geographic_at(row, col, verbose=True)
    grid_locations_of(lat, lon)
    print("")

# Plot Dainyor Bridge streamflow vs. Divide4 and Divide5 TBs/DAV


In [ ]:
# Specify pixel label, lat, lon
#label = "Divide4"
#lat = 36.041678
#lon = 74.880428
label = "Divide5"
lat = 36.163646
lon = 74.848112


In [ ]:
# Get pixel locations in respective grids:
cols, rows = grid_locations_of(lat, lon)
cols, rows

In [ ]:
# AMSR-E data started mid-2002, so open the 36V SIR cubefile for 2003
# and pull the 36V TBs for the 3km pixels

In [ ]:
cubeType = '36V-SIR'
year = 2003
cubeFile = "%s%s%s%s" % (
        "/Users/brodzik/cetb_data/AQUA_AMSRE/N/cubes_UIB/",
        "CETB.cubefile.UIB.AQUA_AMSRE-",
        cubeType,
        "-RSS-v1.2.%4d.TB.nc" % (year))
f = Dataset(cubeFile, "r", "NETCDF4")   
t36V = f.variables['TB'][:]
time = f.variables['time'][:]
f.close()
print("%s, TB=" % label)
ts = t36V[:, rows[0], cols[0]]
print(ts)


In [ ]:
dav = ts[1::2] - ts[0::2]

In [ ]:
fig, ax = plt.subplots(2)
ax[0].plot(ts)
ax[1].plot(dav)


In [ ]:
subULlat, subULlon = 42.6, 152.0
subLRlat, subLRlon = 34.5, 119.0
find_25km_map_corners(subULlat, subULlon, subLRlat, subLRlon)


In [ ]:
test = np.arange(10)
test

In [ ]:
morning = test[0::2]
evening = test[1::2]

In [ ]:
print(evening)
print(morning)

In [ ]:
cols = np.array([0.5, 2.1, 3.8])
cols

In [ ]:
cols + 0.5

In [ ]:
(cols + 0.5).astype('int32')